In [554]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import autograd
from torchvision.datasets import MNIST
from sklearn.naive_bayes import GaussianNB

In [549]:
mnist = MNIST(r'C:\Users\ilyak\Downloads\mnist')
X, y = mnist.data.reshape(60000, 28*28), mnist.targets
X, y = X.to(torch.float64), y.to(torch.int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=77)

In [550]:
dataset = TensorDataset(X_train, y_train)
data_loader = DataLoader(dataset, 10, shuffle=True)

In [552]:
nn = torch.nn.Sequential(torch.nn.Linear(28*28, 10))
nn[0] = nn[0].to(torch.float64)

loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.SGD(nn.parameters(), lr=0.0001)

for epoch in range(1, 16):
    for features, target in data_loader:
        trainer.zero_grad()
        pred = torch.nn.functional.softmax(input=nn(features), dim=1)
        target = torch.nn.functional.one_hot(input=target, num_classes=10).to(torch.float64)
        l = loss(pred, target)
        l.backward()
        trainer.step()
    total_pred = torch.nn.functional.softmax(input=nn(X_train), dim=1)
    l = loss(total_pred, torch.nn.functional.one_hot(input=y_train, num_classes=10).to(torch.float64))
    print(f'Epoch No. {epoch}, loss: {l}')

Epoch No. 1, loss: 1.815486529891147
Epoch No. 2, loss: 1.7740521952999606
Epoch No. 3, loss: 1.7570059505889437
Epoch No. 4, loss: 1.7508375404170984
Epoch No. 5, loss: 1.7431050301254287
Epoch No. 6, loss: 1.7396943897630854
Epoch No. 7, loss: 1.7367405034744148
Epoch No. 8, loss: 1.7371774829720463
Epoch No. 9, loss: 1.7306575699632076
Epoch No. 10, loss: 1.730265287150649
Epoch No. 11, loss: 1.7317122570194152
Epoch No. 12, loss: 1.7266844241012114
Epoch No. 13, loss: 1.7310940692123336
Epoch No. 14, loss: 1.7266993440982408
Epoch No. 15, loss: 1.7276091906961164


In [553]:
pred = torch.nn.functional.softmax(nn(X_test), dim=1).argmax(dim=1)
accuracy_score(pred, y_test)

0.7282666666666666

In [558]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
accuracy_score(nb_model.predict(X_test), y_test)

0.5488666666666666